## Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений


### Решение

In [1]:
import pandas as pd

vlog = pd.read_csv('/home/annindark/_pandas1_datasets/visit_log.csv', delimiter=';')

# Подготовка ф-ции обработки сорса
def SourceType (row):
    if row['traffic_source'] in ('yandex', 'google'):
        return 'organic'
    elif row['traffic_source'] in ('email', 'paid'):
        if row['region'] == 'Russia':
            return 'ad'
        else:
            return 'other'
    else:
        return row['traffic_source']

# Обогащение датафрейма
vlog['source_type'] = vlog.apply(lambda x: SourceType(x), axis=1)

## Задание 2
В файле URLs.txt содержатся url страниц новостного сайта.
Вам необходимо отфильтровать его по адресам страниц с текстами новостей.
Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис.

Выполните следующие действия:

1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

### Решение

In [ ]:
import pandas as pd


urls = pd.read_csv('/home/annindark/_pandas1_datasets/URLs.txt')

urls.loc[urls.url.str.contains(r'/[0-9]{8,8}-')]


## Задание 3

Используйте файл с оценками фильмов ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


### Решение

In [ ]:
import pandas as pd

rate = pd.read_csv('/home/annindark/_pandas1_datasets/ml-latest-small/ratings.csv')

# собираем кол-во оценок, мин и макс дату на юзера
rate = rate.groupby('userId').agg({'timestamp': ['min','max', 'count']})
# Отфильтровываем нужных (боьлше 100 оценок) (положить в предыдущую команде сразу не получилось - ругался, что не знает такие ключи/названия столбцов)
rate = rate.loc[rate['timestamp']['count'] > 100]

# Считаем LT в рамках каждого юзера
rate['lt'] = rate.apply(lambda x: pd.to_datetime(x['timestamp']['max'], unit='s')-pd.to_datetime(x['timestamp']['min'], unit='s'), axis=1)

# Получаем конечный результат
rate['lt'].mean()


## Задание 4

Дана статистика услуг перевозок клиентов компании по типам:
```
rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов
```

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

_Обратите внимание, что в процессе объединения таблиц данные не должны теряться._


### Решение

In [ ]:
import pandas as pd


rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

# Выручка по клиентам
client_base.merge(rzd, on='client_id' , how='left').merge(auto, on='client_id' , how='left').merge(air, on='client_id' , how='left').iloc[:,[0,2,3,4]].groupby('client_id').agg(sum)

# Не уверена, что корректно поняла последнюю часть задания 

# Выручка по адресам
client_base.merge(rzd, on='client_id' , how='left').merge(auto, on='client_id' , how='left').merge(air, on='client_id' , how='left').iloc[:,[1,2,3,4]].groupby('address').agg(sum)

# Выручка по клиентам с адресами
client_base.merge(rzd, on='client_id' , how='left').merge(auto, on='client_id' , how='left').merge(air, on='client_id' , how='left').groupby(['client_id', 'address']).agg(sum)